In [2]:
from qutip import *
from qutip.measurement import measure, measurement_statistics
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import time
from tqdm.notebook import tqdm

def plot_wigner(rho, fig=None, ax=None):
    """
    Plot the Wigner function and the Fock state distribution given a density matrix for
    a harmonic oscillator mode.
    """
    
    if fig is None or ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(4,4))

    if isket(rho):  # ket状態を密度関数にする（必要かわからん）
        rho = ket2dm(rho)
    
    scale = np.sqrt(2)
    xvec = np.linspace(-5*scale,5*scale,100)

    W = wigner(rho, xvec, xvec)
    wlim = abs(W).max()

    ax.contourf(xvec/scale, xvec/scale, W, 60, norm=mpl.colors.Normalize(-wlim,wlim), cmap=mpl.cm.get_cmap('RdBu'))
    ax.set_xlabel('q', fontsize=16)
    ax.set_ylabel('p', fontsize=16)
    #ax.set_title()
    fig.tight_layout
    
    return fig, ax

In [3]:
def rotation(dim, phi):
    op = 1j * phi * adag * a
    return op.expm()

def kerr(dim, kappa):
    n = adag * a
    op = 1j * kappa * n * n
    return op.expm()

In [13]:
def variational_quantum_circuit(params):
    
    # Gate layer: D-R-S-R-K
    def layer(i, phi):
        s = time.perf_counter()
        D = displace(dim, params[i*7+0]*np.exp(1j*params[i*7+1]))
        R1 = rotation(dim, params[i*7+2])
        S = squeeze(dim, params[i*7+3]*np.exp(1j*params[i*7+4]))
        R2 = rotation(dim, params[i*7+5])
        K = kerr(dim, params[i*7+6])
        quantum_state = K * D * R2 * S * R1 * phi
        e = time.perf_counter()  
        return quantum_state, e-s
    
    # construct the circuit
    state = fock(dim, 0)
    for i in range(depth):
        state, t = layer(i, state)
    
    #print('time',e-s)
    return state, t

In [14]:
dim = 30
depth = 8

a = destroy(dim)
adag = a.dag()

t = np.zeros(100)
for i in tqdm(range(100)):
    params = np.random.normal(0, 0.1, [7*depth]) # 最初のパラメータ
    state, t[i] = variational_quantum_circuit(params)
print('avg',np.sum(t/100))

  0%|          | 0/100 [00:00<?, ?it/s]

avg 0.0034226740000303834


In [17]:
dim = 30
# 単純な行列の積
state1_real = np.random.rand(dim, dim)
state1_imag = np.random.rand(dim, dim)
state1_np = state1_real + 1j * state1_imag
state1_qobj = Qobj(state1_np)

state2_real = np.random.rand(dim, 1)
state2_imag = np.random.rand(dim, 1)
state2_np = state2_real + 1j * state2_imag
state2_qobj = Qobj(state2_np)

itr = 16

s_np = time.perf_counter()
for i in range(itr):
    a_np = state1_np @ state2_np
e_np = time.perf_counter()

s_qobj = time.perf_counter()
for i in range(itr):
     a_qobj = state1_qobj * state2_qobj
e_qobj = time.perf_counter()

print('time_np', e_np-s_np)
print('time_qobj', e_qobj-s_qobj)

time_np 7.609999920532573e-05
time_qobj 0.0007089999999152496


In [9]:
dim = 30
# 指数行列の計算
state1_real = np.random.rand(dim, dim)
state1_imag = np.random.rand(dim, dim)
state1_np = state1_real + 1j * state1_imag
state1_qobj = Qobj(state1_np)

itr = 100

s_qobj = time.perf_counter()
for i in range(itr):
    a_qobj = state1_qobj.expm()
e_qobj = time.perf_counter()

s_np = time.perf_counter()
for i in range(itr):
    a_np = expm(state1_np)
e_np = time.perf_counter()

print('time_np', e_np-s_np)
print('time_qobj', e_qobj-s_qobj)

time_np 0.04111600000032922
time_qobj 0.07014329999947222
